In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks


In [2]:

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar100.load_data()

x_train.shape, x_test.shape

169001437/169001437 [==============================] - 13s 0us/step


((50000, 32, 32, 3), (10000, 32, 32, 3))

In [3]:
# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0



In [4]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

In [5]:
# Define the model
model = models.Sequential()

# Data augmentation layer
model.add(layers.InputLayer(input_shape=(32, 32, 3)))
model.add(data_augmentation)

# First Convolutional Block
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

In [6]:
# Second Convolutional Block
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

# Third Convolutional Block
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

In [7]:
# Flatten the output and add Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='softmax'))  # 100 classes for CIFAR-100

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)       

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
import os
checkpoint_dir = '/content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Set up model checkpointing
checkpoint = callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'cifar100_cnn_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


In [11]:
# Train the model with checkpointing
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=128,
    validation_data=(x_test, y_test),
    callbacks=[checkpoint]
)


Epoch 1/50
391/391 [==============================] - ETA: 0s - loss: 4.2610 - accuracy: 0.0833
Epoch 1: val_accuracy improved from -inf to 0.05390, saving model to /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/cifar100_cnn_epoch_01_val_acc_0.05.h5
391/391 [==============================] - 23s 31ms/step - loss: 4.2610 - accuracy: 0.0833 - val_loss: 4.5161 - val_accuracy: 0.0539
Epoch 2/50
  1/391 [..............................] - ETA: 9s - loss: 3.7242 - accuracy: 0.1953

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


390/391 [============================>.] - ETA: 0s - loss: 3.5179 - accuracy: 0.1661
Epoch 2: val_accuracy improved from 0.05390 to 0.18830, saving model to /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/cifar100_cnn_epoch_02_val_acc_0.19.h5
391/391 [==============================] - 9s 23ms/step - loss: 3.5182 - accuracy: 0.1661 - val_loss: 3.3658 - val_accuracy: 0.1883
Epoch 3/50
391/391 [==============================] - ETA: 0s - loss: 3.1665 - accuracy: 0.2233
Epoch 3: val_accuracy improved from 0.18830 to 0.29750, saving model to /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/cifar100_cnn_epoch_03_val_acc_0.30.h5
391/391 [==============================] - 10s 25ms/step - loss: 3.1665 - accuracy: 0.2233 - val_loss: 2.7838 - val_accuracy: 0.2975
Epoch 4/50
390/391 [============================>.] - ETA: 0s - loss: 2.9628 - accuracy: 0.2610
Epoch 4: val_accuracy did not improve from 0.29750
391/391 [==============================] - 9s 24m

In [13]:
import os
import re

# Get a list of all checkpoint files
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('cifar100_cnn_epoch')]

# Extract the validation accuracy from the filenames and find the best one
best_val_acc = 0
best_model_path = ""

val_acc_pattern = re.compile(r"_val_acc_([0-9]+\.[0-9]+).h5")
for checkpoint_file in checkpoint_files:
    match = val_acc_pattern.search(checkpoint_file)
    if match:
        val_acc = float(match.group(1))
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = os.path.join(checkpoint_dir, checkpoint_file)

print(f"Best model: {best_model_path} with validation accuracy: {best_val_acc}")


Best model: /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/cifar100_cnn_epoch_42_val_acc_0.56.h5 with validation accuracy: 0.56


In [14]:
from tensorflow.keras.models import load_model

# Load the best model
best_model = load_model(best_model_path)

# Evaluate the best model
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


313/313 - 1s - loss: 1.6110 - accuracy: 0.5579 - 1s/epoch - 4ms/step
Test accuracy: 0.5579000115394592


In [17]:
# Save the best model as a new file
best_model_save_path = os.path.join(checkpoint_dir, 'best_model.h5')
best_model.save(best_model_save_path)
print(f"Best model saved to: {best_model_save_path}")

Best model saved to: /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/best_model.h5
Best model saved to: /content/drive/MyDrive/DL Practical Assignments/cifar100_checkpoints/best_model.h5
